In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

%load_ext autoreload
%autoreload 2

In [ ]:
# Parameters
R = 5
omega_1 = 0.5  # leader speed [rad/s]
N = 100  # number of timesteps
T = np.linspace(0, 10, N)  # time vector
theta_sigma = 0.01  # standard deviation of theta dynamics noise

# Initial conditions
theta_1 = np.pi  # leader state
theta_2 = 0  # follower state

In [ ]:
# Dynamics matrix
dt = 0.1
K = 1
A = np.array([[1, dt, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, dt],
              [K, 1, -K, 0]])

# Bias vector
b = np.array([[0],
              [0],
              [0],
              [-K*np.pi]])

x = np.array([[theta_1],
              [omega_1],
              [theta_2],
              [omega_1]])

# Simulate dynamics
X = np.zeros((N, 4))
for i in range(N):
    v_1 = np.random.normal(0, theta_sigma)
    v_2 = np.random.normal(0, theta_sigma)
    x = A @ x + b
    x[0] += v_1
    x[2] += v_2
    X[i] = x[:,0]

In [ ]:
X

In [ ]:
from multirtd.zonotope import Zonotope

In [ ]:
# Reachability
# State zonotope
c_x = np.array([[theta_1],
              [omega_1],
              [theta_2],
              [omega_1]])
G_x = np.zeros((4,1))
X = Zonotope(c_x, G_x)

# Noise zonotope
# (3-sigma confidence interval)
c_v = np.zeros((4,1))
G_v = np.array([[3*theta_sigma, 0],
                [0, 0],
                [0, 3*theta_sigma],
                [0, 0]])
V = Zonotope(c_v, G_v)

Xs = []
for i in range(N):
    X = A * X + b + V
    Xs.append(X)

In [ ]:
print(np.sum(np.abs(Xs[-1].G), axis=1))

In [ ]:
# Simulate leader dynamics
x_1 = R * np.cos(omega_1 * T + theta_1)
y_1 = R * np.sin(omega_1 * T + theta_1)

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = ax.plot([], [], lw=2)

def init():
    ax.set_xlim(-1.5*R, 1.5*R)
    ax.set_ylim(-1.5*R, 1.5*R)
    return ln,

def update(i):
    t = 0.1*i
    R = 5
    omega_1 = 0.2
    theta_1 = 0

    x_1 = R * np.cos(omega_1 * t + theta_1)
    y_1 = R * np.sin(omega_1 * t + theta_1)
    xdata.append(x_1)
    ydata.append(y_1)
    ln.set_data(xdata, ydata)
    return ln,

ani = FuncAnimation(fig, update, frames=N, interval=20,
                    init_func=init, blit=True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = ax.plot([], [], 'ro')

def init():
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1, 1)
    return ln,

def update(frame):
    xdata.append(frame)
    ydata.append(np.sin(frame))
    ln.set_data(xdata, ydata)
    return ln,

ani = FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128),
                    init_func=init, blit=True)
plt.show()